In [1]:
import pandas as pd
import numpy as np
import os
from glob import glob
from functools import reduce
import tqdm.notebook as tq

In [2]:
def df_from_list(df_list: list, merge_col='date') -> pd.DataFrame:
    """Function merge DataFrames from list to the single frame

    Args:
        df_list (list): List of dataframes
        merge_col (str, optional): column on which merge will be perfomed.
                                   Defaults to 'date'.

    Returns:
        pd.DataFrame: Merged dataframe
    """
    return reduce(lambda left, right: pd.merge(left, right, on=merge_col), df_list)


In [3]:
path_to_meteo = 'S:/education/HSI/aspirantura/Dissertation/conus_data/WS_meteo/'
path_to_geo = 'S:/education/HSI/aspirantura/Dissertation/conus_data/featureXtractor/'
save_path = 'S:/education/HSI/aspirantura/Dissertation/conus_data/geo_met/'
os.makedirs(save_path, exist_ok=True)

In [4]:
all_meteo = glob(path_to_meteo + '*.csv')

In [5]:
# merge HydroATLAS data to one dataframe
test_geo = df_from_list(df_list=[pd.read_csv(path, sep=';') 
                                 for path in glob(path_to_geo + '*.csv')],
                        merge_col='gauge_id')
# simplify comprassion with modify of gauge_id column
test_geo['gauge_id'] = ['0' + str(gauge) if len(str(gauge)) != 8
                        else str(gauge)
                        for gauge in test_geo['gauge_id']]

In [6]:
# Necessary columns to work with time-varying variables
monthes = ['01', '02', '03', '04', '05', '06',
           '07', '08', '09', '10', '11', '12']

time_varying_vars = [item for sublist in [['tmp_dc_s{}'.format(i)
                                           for i in monthes],
                                          ['pre_mm_s{}'.format(i)
                                           for i in monthes],
                                          ['pet_mm_s{}'.format(i)
                                           for i in monthes],
                                          ['aet_mm_s{}'.format(i)
                                           for i in monthes],
                                          ['cmi_ix_s{}'.format(i)
                                           for i in monthes],
                                          ['snw_pc_s{}'.format(i)
                                           for i in monthes],
                                          ['swc_pc_s{}'.format(i)
                                           for i in monthes]]
                     for item in sublist]


In [7]:
for meteo_file in tq.tqdm(all_meteo):
    # loop through all files to generete new one
    gauge = meteo_file.split('\\')[1][:-4]
    gauge_geo = test_geo.loc[np.where(test_geo['gauge_id'] == gauge)]
    # split variables for static and dynamic (monthly varying)
    varying_vars = gauge_geo[time_varying_vars]
    static_vars = gauge_geo[gauge_geo.columns.difference(time_varying_vars)]
    # read meteo file. Later it will be modified for final output
    gauge_meteo = pd.read_csv(meteo_file)
    # expand values to fullfill final file
    expanded_values = [[value]*len(gauge_meteo)
                       for value in static_vars.values[0]]
    # assign to final frame static variables
    for i, column in enumerate(static_vars.columns):
        gauge_meteo.loc[:, column] = expanded_values[i]
    # create columns for varying variables
    unique_varies = np.unique([col[:-2] for col in varying_vars.columns])
    # assert to final frame dynamic variables
    for i, date in enumerate(gauge_meteo['date']):
        month = date.split('-')[1]
        for variable in unique_varies:
            gauge_meteo.loc[i, variable] = varying_vars[variable + month].to_list()
    gauge_meteo.to_csv(save_path + f'{gauge}_met_geo.csv',
                       index=False)


  0%|          | 0/671 [00:00<?, ?it/s]